In [1]:
import pandas as pd
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.grid_search import GridSearchCV
import time

In [2]:
dframe = pd.read_csv('../data/transformed.csv')

In [3]:
X = dframe.drop('OutcomeType', axis=1)
X.columns

Index([u'AgeuponOutcome', u'doy', u'Name_hasName', u'Name_noName',
       u'AnimalType_Cat', u'AnimalType_Dog', u'SexuponOutcome_Intact Female',
       u'SexuponOutcome_Intact Male', u'SexuponOutcome_Neutered Male',
       u'SexuponOutcome_Spayed Female', u'Breed_Mix', u'Breed_Pure',
       u'dow_Fri', u'dow_Mon', u'dow_Sat', u'dow_Sun', u'dow_Thurs',
       u'dow_Tues', u'dow_Wed', u'month_April', u'month_Aug', u'month_Dec',
       u'month_Feb', u'month_Jan', u'month_July', u'month_June',
       u'month_March', u'month_May', u'month_Nov', u'month_Oct',
       u'month_Sept'],
      dtype='object')

In [4]:
y = dframe['OutcomeType']

In [5]:
rfc = RandomForestClassifier(class_weight="balanced")

In [6]:
## not sure why this isn't working

# kfold = cross_validation.KFold(len(X), n_folds=5)

# [rfc.fit(X[train], y[train]).score(X[test], y[test]) for train, test in kfold]

In [7]:
# cross_val_score() does the kfold cross val in previous cell
# scores are as good as the best scores on Kaggle; random forest classifier is a good choice
cross_validation.cross_val_score(rfc, X, y, cv=5, n_jobs=-1, scoring="recall")

/Users/pc3sq/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/Users/pc3sq/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1

array([ 0.5902439 ,  0.59211553,  0.59312768,  0.59031439,  0.590625  ])

In [8]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.25, random_state=7)

In [9]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
probas = rfc.predict_proba(X_test)

Notice that probas has 5 columns. This is due to the fact that we have 5 outcomes. For each observation, we pick the outcome that has the highest probability out of the 5 probabilities we calculate.

In [11]:
probas.shape

(6403, 5)

In [12]:
log_loss(y_test, probas)

4.4182396616559689

In [13]:
y_test.unique()

array(['Return_to_owner', 'Transfer', 'Adoption', 'Euthanasia', 'Died'], dtype=object)

In [14]:
pr

NameError: name 'pr' is not defined

In [ ]:
# easy_cf(y_test, preds)

In [ ]:
import 
def easy_cf(actuals, predicted):
    cf_matrix = confusion_matrix(actuals, predicted)

    '''
    true positive (TP); eqv. with hit
    false positive (FP); eqv. with false alarm, Type I error
    true negative (TN); eqv. with correct rejection
    false negative (FN); eqv. with miss, Type II error
    '''

    tn = cf_matrix[0, 0]
    fn = cf_matrix[1, 0]
    tp = cf_matrix[1, 1]
    fp = cf_matrix[0, 1]
    total = len(actuals)

    pred_pos = tp + fp
    pred_neg = tn + fn

    act_pos = tp + fn
    act_neg = tn + fp

    accuracy = (tp + tn) / float(total)

    if pred_pos == 0:
        recall = "No predicted +"
        precision = "No predicted +"
    else:
        '''
        Recall, or sensitivity (true positive rate), quantifies the avoiding of false negatives.
        '''
        recall = float(tp)/act_pos

        '''
        Precision, or positive predictive value (PPV).
        If my test predicts positive, how certain can I be that it is a true positive?
        '''
        precision = float(tp)/pred_pos


    if pred_neg == 0:
        specificity = "No predicted -"
    else:
        '''
        Specificity, or true negative rate, quantifies the avoiding of false positives.
        '''
        specificity = float(tn)/act_neg


    print "True Positive is:", tp
    print "False Positive is:", fp
    print "True Negative is:", tn
    print "False Negative is:", fn
    print

    print "Predicting + and - correctly"
    print "Acccuracy:", accuracy
    print

    print "How much can I count on my + prediction?"
    print "Precision:", precision
    print

    print "Capturing actual + and avoiding FN"
    print "Sensitivity / Hit Rate / Recall:", recall
    print

    print "Capturing actual - and avoiding FP"
    print "Specificity / True Negative Rate:", specificity
    print




In [ ]:
def feat_importance(rfc_model, X_test):
    importances = rfc_model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rfc_model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    columns = np.array([   'length', 'percent_nouns',
        'subjectivity',      'polarity',    'has_reason', 'has_therefore',
          'has_reject',   'has_applied',  'has_standard',      'has_fact',
           'has_argue',           'act',        'action',       'appeals',
                'case',         'cases',    'certiorari',       'circuit',
               'claim',        'claims',         'court',        'courts',
                 'did',      'district',          'does',      'evidence',
             'federal',         'filed',       'general',          'held',
         'information',        'joined',      'judgment',          'jury',
                 'law',         'legal',          'make',      'official',
             'opinion',    'petitioner',   'petitioners',            'pp',
         'respondents',          'rule',          'site',         'state',
              'states',       'supreme',           'tax',         'trial',
              'united'])

    col_importance = columns[indices]


    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_test.shape[1]):
        print("%d. feature %d: %s (%f) " % (f + 1, indices[f], col_importance[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.title("Feature importances")
    plt.bar(range(X_test.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X_test.shape[1]), indices)
    plt.xlim([-1, X_test.shape[1]])
    plt.show()

    #Plot feature importances of the forest in seaborn
    sns.set_style("whitegrid")


# Gradient Boosting Classifier

In [ ]:
# trying gradient boost model
gbm = GradientBoostingClassifier(n_estimators=100,random_state=7).fit(X_train,y_train)

In [ ]:
gbm_probas = gbm.predict_proba(X_test)

In [ ]:
log_loss(y_test, gbm_probas)

In [ ]:
X_train.shape

It makes sense that a gradient boosting classifier works very well because with each new iteration of the estimator, it is placing more weight on the misclassified classes from the previous estimator. Because more weight is placed on previous misclassifications, GBC inherently deals with class imbalance.

# GridSearch w/ Gradient Boosting Classifier

In [ ]:
# gradient boosting hyperparameters available for tweaking
gbm.get_params().keys()

In [ ]:
dframe[(dframe['OutcomeType']=='Died') | (dframe['OutcomeType']=='Euthanasia')].shape[0]

In [ ]:
# number of features
len(X.columns)

In [ ]:
gsearch_gbc = GradientBoostingClassifier(learning_rate=0.1, 
                                         max_depth=8, 
                                         max_features="auto", 
                                         min_samples_leaf=250) # 1623 died or euthansized

In [ ]:
# first set of params
param_test1 = {"n_estimators": [50,80],
               "min_samples_split": [500, 1000]}

# run grid search
gsearch1 = GridSearchCV(gsearch_gbc, param_grid=param_test1, n_jobs=-1)
start1 = time.time()
gsearch1.fit(X_train, y_train)
end1 = time.time()

In [ ]:
gsearch1_probas = gsearch1.predict_proba(X_test)

In [ ]:
log_loss(y_test, gsearch1_probas)

In [ ]:
end1 - start1 # takes just over a minute to run params_test1

In [ ]:
gsearch1.best_params_

In [ ]:
# second set of params
param_test2 = {"n_estimators": [120, 200],
               "min_samples_split": [500, 1000]}

# run grid search
gsearch2 = GridSearchCV(gsearch_gbc, param_grid=param_test2, n_jobs=-1)
start2 = time.time()
gsearch2.fit(X_train, y_train)
end2 = time.time()

In [ ]:
end2 - start2 # takes just over 2 minute to run params_test2

In [ ]:
gsearch2_probas = gsearch2.predict_proba(X_test)

In [ ]:
log_loss(y_test, gsearch2_probas)

In [ ]:
gsearch2.best_params_

In [ ]:
# third set of params
gsearch_gbc3 = GradientBoostingClassifier(max_depth=8, 
                                         min_samples_leaf=1000) # 1623 died or euthansized

param_test3 = { "learning_rate": [0.1, 0.05, 0.01],
                "max_features": ["sqrt", 3, 10]}

gsearch3 = GridSearchCV(gsearch_gbc3, param_grid=param_test3, n_jobs=-1)
start3 = time.time()
gsearch3.fit(X_train, y_train)
end3 = time.time()

In [ ]:
end3 - start3

In [ ]:
gsearch3.best_params_

In [ ]:
gsearch3_probas = gsearch3.predict_proba(X_test)

In [ ]:
log_loss(y_test, gsearch3_probas)